In [5]:
import os 
print( os.getcwd() )
# change working directory
os.chdir( "/var/genie-in-the-box/src" )
print( os.getcwd() )
import lib.utils.util as du

/var/genie-in-the-box/src/ephemera/notebooks
/var/genie-in-the-box/src


In [69]:
# associate commands with files containing examples
browser_commands = {
    "search new tab": "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt",
    "search current tab": "/src/ephemera/prompts/data/synthetic-data-search-in-current-tab.txt",
    "search google new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-new-tab.txt",
    "search google current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-current-tab.txt",
    "search google scholar new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-new-tab.txt",
    "search google scholar current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-current-tab.txt",
}

In [70]:
for command in browser_commands.keys():
    print( os.path.exists( du.get_project_root() + browser_commands[ command ] ) )

True
True
True
True
True
True


In [42]:
raw_lines = du.get_file_as_list( du.get_project_root() + "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt", clean=True )[ 0:100 ]
raw_lines[ 0:10 ]

['SEARCH_TERMS new tab search',
 'New tab, search SEARCH_TERMS',
 'Open new tab, search for SEARCH_TERMS',
 'Pull up SEARCH_TERMS in another tab',
 'Find SEARCH_TERMS in a new tab',
 'Hunt for SEARCH_TERMS, new tab',
 'Display SEARCH_TERMS results in new tab',
 'Do a search for SEARCH_TERMS and show results in another tab',
 'Execute SEARCH_TERMS search in a new tab',
 'Seek SEARCH_TERMS in a fresh tab']

In [58]:
def get_search_terms( requested_length ):
    
    # Load search terms file
    search_terms = du.get_file_as_list( du.get_project_root() + "/src/ephemera/prompts/data/search-terms.txt", lower_case=False, clean=True, randomize=True )
    
    # If we don't have enough search terms, append copies of the search term list until we do
    while requested_length > len( search_terms ):
        search_terms += search_terms
        
    # Truncate the search terms list to equal the requested len
    search_terms = search_terms[ :requested_length ]
    
    return search_terms

In [60]:
search_terms = get_search_terms( len( raw_lines ) )
search_terms[ 0:10]

['famous historical events',
 'how to make pizza dough',
 'top 10 action movies',
 'current news in business',
 'Java for beginners',
 'what is the capital of Brazil?',
 'funny jokes for kids',
 'current news in tech',
 'how to overcome anxiety',
 'need new summer clothes']

In [95]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
command_choices 

"'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'"

In [103]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
prompt_template = """
Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

The raw human voice command transcription is in simple XML: 
{human_says}
The standardized command that you translate MUST be returned in simple, well-formed XML:
{response_format}
"""

human_says_template = """
<human>
    <voice-command>{voice_command}</voice-command>
</human>
"""

response_format = """
<response>
    <browser-command></browser-command>
    <args></args>
</response>
"""

answer_template = """
<response>
    <browser-command>{browser_command}</browser-command>
    <args>{args}</args>
</response>
"""

In [104]:
prompt_template

"\nYour job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.\n\nYou will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.\n\nHint: Anything that isn't a part of the command itself should be treated as arguments related to the command.\n\nThe raw human voice command transcription is in simple XML: \n{human_says}\nThe standardized command that you translate MUST be returned in simple, well-formed XML:\n{response_format}\n"

In [105]:
prompts   = []
answers   = []
responses = []

for browser_command in browser_commands.keys():
    
    du.print_banner( browser_command, prepend_nl=True, end="\n" )
    
    raw_lines = du.get_file_as_list( du.get_project_root() + browser_commands[ browser_command ], clean=True )
    for line in raw_lines[ 0:100 ]: #[ 0:2 ]:#
        
        # get newly randomized search terms on every iteration
        for search in get_search_terms( len( raw_lines ) ):#[ 0:10 ]: #[ 0:2]:#:
            
            voice_command = line.replace( "SEARCH_TERMS", search )
            # print( voice_command )
            human_says    = human_says_template.format( voice_command=voice_command )
            prompt        = prompt_template.format( human_says=human_says, response_format=response_format, command_choices=command_choices )
            response      = answer_template.format( browser_command=browser_command, args=search )
    
            prompts.append( prompt )
            answers.append( response )
    
            # du.print_banner( "Prompt" )
            # print( prompt )
            # du.print_banner( "Response" )
            # print( response )
            print( ".", end="" )
        
    print()


------------------------------------------------------------------------------------------------------------------------
- search new tab
------------------------------------------------------------------------------------------------------------------------
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [106]:
idx = 0

for line in prompts[ 0 ].split( "\n" ):
    print( line )
    
for line in answers[ idx ].split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

The raw human voice command transcription is in simple XML: 

<human>
    <voice-command>I need hiking gear new tab search</voice-command>
</human>

The standardized command that you translate MUST be returned in simple, well-formed XML:

<response>
    <browser-command></browser-command>
    <args></args>
</response>



<response>
    <browser-command>search new tab</browser-comm

In [107]:
len( answers )

120000

In [108]:
len( prompts )

120000

In [109]:
for line in prompts[ 0 ].split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

The raw human voice command transcription is in simple XML: 

<human>
    <voice-command>I need hiking gear new tab search</voice-command>
</human>

The standardized command that you translate MUST be returned in simple, well-formed XML:

<response>
    <browser-command></browser-command>
    <args></args>
</response>



In [86]:
for line in answers[ 0 ].split( "\n" ):
    print( line )


<response>
    <browser-command>search new tab</browser-command>
    <args>why onions make us cry</args>
</response>
